# Prevendo roubo de celulares em SP

## Baixando biblioteca necessária
`pip install xlwt`

`conda install xlwt`

## Importando bibliotecas Pandas e Numpy

In [1]:
import pandas as pd
import numpy as np

## Abrindo arquivo XLS disponível no SSP-SP

### Os arquivos possuem um erro, sendo impossível abrir o arquivo
*XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'A\x00N\x00O\x00_\x00'*

A solução foi encontrada nesse artigo do [Medium](https://medium.com/@jerilkuriakose/recover-corrupt-excel-file-xls-or-xlsx-using-python-2eea6bb07aae)

#### Apenas para resolver o problema de formato
Você pode até salvar o arquivo como .csv, mas a leitura pode informar algum erro

In [ ]:
from __future__ import unicode_literals
# Used to save the file as excel workbook
# Need to install this library
from xlwt import Workbook
# Used to open to corrupt excel file
import io

filename = 'datasets/DadosBO_2020_8.xls'
# Opening the file using 'utf-16' encoding
file = io.open(filename, "r", encoding="utf-16-le")
data = file.readlines()

# Creating a workbook object
xldoc = Workbook()
# Adding a sheet to the workbook object
sheet = xldoc.add_sheet("Sheet1", cell_overwrite_ok=True)
# Iterating and saving the data to sheet
for i, row in enumerate(data):
    # Two things are done here
    # Removeing the '\n' which comes while reading the file using io.open
    # Getting the values after splitting using '\t'
    for j, val in enumerate(row.replace('\n', '').split('\t')):
        sheet.write(i, j, val)
    
# Saving the file as an excel file
xldoc.save('datasets/file.xls')

#### Converter xls para csv

In [ ]:
import xlrd
import csv

def xls_to_csv():
    wb = xlrd.open_workbook('datasets/file.xls')
    sh = wb.sheet_by_name('Sheet1')
    with open('dataset.csv', 'w') as f:
        wr = csv.writer(f, quoting=csv.QUOTE_ALL)

        for rownum in range(sh.nrows):
            wr.writerow(sh.row_values(rownum))
#https://stackoverflow.com/questions/9884353/xls-to-csv-converter
xls_to_csv()

## Carregando o dataset

In [29]:
#dataset = pd.read_excel("DadosBO_2020_7.xls", engine='xlrd')
from datetime import datetime
dateparse = lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M:%S')

columns = ['ANO_BO', 'NUM_BO', 'BO_INICIADO', 'BO_EMITIDO', 'DATAOCORRENCIA', 'HORAOCORRENCIA', 'PERIDOOCORRENCIA', 'DATACOMUNICACAO', 'DATAELABORACAO', 'BO_AUTORIA', 'FLAGRANTE', 'LOGRADOURO', 'NUMERO', 'BAIRRO', 'CIDADE', 'UF', 'LATITUDE', 'LONGITUDE', 'DESCRICAOLOCAL', 'DESCR_TIPO_VEICULO', 'SOLUCAO', 'DELEGACIA_NOME', 'DELEGACIA_CIRCUNSCRICAO', 'ESPECIE', 'RUBRICA', 'DESDOBRAMENTO', 'STATUS', 'QUANT_CELULAR', 'MARCA_CELULAR']

column_dates = ['BO_INICIADO', 'BO_EMITIDO', 'DATACOMUNICACAO', 'DATAELABORACAO']

dtypes = {
    'ANO_BO': np.int32,
    'NUM_BO': np.int64,
    'QUANT_CELULAR': pd.Int16Dtype(),
    'LATITUDE': np.float32,
    'LONGITUDE': np.float32 
}

dataset = pd.read_excel("datasets/file.xls", usecols=columns, dtype=dtypes, thousands=',')

In [31]:
dataset

,ANO_BO,NUM_BO,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,...,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
0,2020,1069122,01/08/2020 00:09:32,01/08/2020 00:07:52,28/07/2020,11:45,PELA MANHÃ,31/07/2020,01/08/2020 00:09:32,Desconhecida,...,BO PARA REGISTRO,DELEGACIA ELETRONICA,62º D.P. ERMELINO MATARAZZO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,<NA>,LG
1,2020,1069823,31/07/2020 23:25:40,01/08/2020 00:10:55,31/07/2020,12:53,A TARDE,31/07/2020,31/07/2020 23:25:40,Desconhecida,...,ENCAMINHAMENTO DP ÁREA DO FATO,DELEGACIA ELETRONICA,05º D.P. ACLIMACAO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - TRANSEUNTE,NaN,Consumado,NaN,<NA>,APPLE
2,2020,1069786,31/07/2020 23:12:57,01/08/2020 00:16:51,31/07/2020,16:55,A TARDE,31/07/2020,31/07/2020 23:12:57,Desconhecida,...,ENCAMINHAMENTO DP ÁREA DO FATO,DELEGACIA ELETRONICA,54º D.P. CID. TIRADENTES,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - TRANSEUNTE,NaN,Consumado,NaN,<NA>,SAMSUNG
3,2020,1495,31/07/2020 21:34:49,01/08/2020 00:17:43,31/07/2020,21:00,A NOITE,31/07/2020,31/07/2020 21:34:49,Conhecida,...,BO PARA FLAGRANTE,DEL.SEC.JACAREI PLANTÃO,02º D.P. JACAREI,Ato infracional,A.I.-Roubo (art. 157) - VEICULO,NaN,Consumado,NaN,1,Motorola
4,2020,1495,31/07/2020 21:34:49,01/08/2020 00:17:43,31/07/2020,21:00,A NOITE,31/07/2020,31/07/2020 21:34:49,Conhecida,...,BO PARA FLAGRANTE,DEL.SEC.JACAREI PLANTÃO,02º D.P. JACAREI,Ato infracional,A.I.-Roubo (art. 157) - VEICULO,NaN,Consumado,NaN,1,SAMSUNG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13527,2020,5342,31/08/2020 23:36:15,31/08/2020 23:48:52,31/08/2020,22:00,A NOITE,31/08/2020,31/08/2020 23:36:15,Desconhecida,...,APRECIAÇÃO DO DELEGADO TITULAR,DEL.SEC.RIO CLARO PLANTÃO,03º D.P. RIO CLARO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,1,MOTOROLA
13528,2020,4656,31/08/2020 23:31:17,31/08/2020 23:52:29,31/08/2020,19:10,A NOITE,31/08/2020,31/08/2020 23:31:17,Desconhecida,...,BO PARA INVESTIGAÇÃO,101º D.P. JDIM IMBUIAS,101º D.P. JDIM IMBUIAS,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - TRANSEUNTE,NaN,Consumado,NaN,1,Samsung
13529,2020,1255538,31/08/2020 21:10:54,31/08/2020 23:55:22,31/08/2020,15:00,A TARDE,31/08/2020,31/08/2020 21:10:54,Desconhecida,...,BO PARA REGISTRO,DELEGACIA ELETRONICA,03º D.P. CUBATÃO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - ESTABELECIMENTO-OUTROS,NaN,Consumado,NaN,1,MOTOROLA
13530,2020,1255308,31/08/2020 23:57:57,31/08/2020 23:57:52,31/08/2020,19:30,A NOITE,31/08/2020,31/08/2020 23:57:57,Desconhecida,...,BO PARA REGISTRO,DELEGACIA ELETRONICA,77º D.P. SANTA CECILIA,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,<NA>,XIAOMI


#### Antes da eliminação das duplicatas

In [9]:
before_drop = dataset.shape[0]
print("Quantidade de tuplas: ",dataset.shape[0])

Quantidade de tuplas:  13532


#### Eliminando duplicatas de acordo com [Metodologia](https://drive.google.com/file/d/1_CBHVulHPzLzRyCNpxGXSECVDV-u21QH/view)

In [10]:
dataset.drop_duplicates(subset=['DELEGACIA_NOME', 'ANO_BO', 'NUM_BO'], inplace=True)

In [32]:
dataset['HORARIOOCORRENCIA'] = pd.to_datetime(dataset.DATAOCORRENCIA + ' ' + dataset.HORAOCORRENCIA)

#### Após remoção das duplicatas

In [12]:
after_drop = dataset.shape[0]
print("Quantidade de tuplas: ",dataset.shape[0])

Quantidade de tuplas:  11071


#### Percentual de redução no dataset:

In [ ]:
print("Redução de: ", 100-(after_drop/before_drop)*100)

#### Maioria das ocorrências não são flagrante

In [ ]:
print('Quantidade de ocorrências que é flagrante: ',dataset[dataset["FLAGRANTE"]=="Sim"].shape[0])

In [ ]:
dataset.dtypes

#### Converter coluna com horario da ocorrencia para segundos

In [ ]:
dataset['timestamp_ocorrencia'] = dataset['HORARIOOCORRENCIA'].dt.hour*3600 + dataset['HORARIOOCORRENCIA'].dt.minute*60 + dataset['HORARIOOCORRENCIA'].dt.second

#### Horário médio de ocorrência de crimes por turno(manhã, tarde, noite, madrugada)

In [ ]:
import datetime
manha = dataset[dataset['PERIDOOCORRENCIA']=="PELA MANHÃ"]['timestamp_ocorrencia'].mean()
str(datetime.timedelta(seconds=manha))

In [ ]:
tarde = dataset[dataset['PERIDOOCORRENCIA']=="A TARDE"]['timestamp_ocorrencia'].mean()
str(datetime.timedelta(seconds=tarde))

In [ ]:
noite = dataset[dataset['PERIDOOCORRENCIA']=="A NOITE"]['timestamp_ocorrencia'].mean()
str(datetime.timedelta(seconds=noite))

In [ ]:
madrugada = dataset[dataset['PERIDOOCORRENCIA']=="DE MADRUGADA"]['timestamp_ocorrencia'].mean()
str(datetime.timedelta(seconds=madrugada))

In [ ]:
len(dataset['timestamp_ocorrencia'].unique())

#### Top 5 horários com maior incidência de crimes por turno

In [ ]:
dataset[dataset['PERIDOOCORRENCIA']=="A NOITE"].nlargest(5, "timestamp_ocorrencia")

#### Média de dias para comunicar a policia sobre o roubo

In [17]:
dataset['DATAOCORRENCIA'] = dataset['DATAOCORRENCIA'].astype('datetime64')
dataset['DATACOMUNICACAO'] = dataset['DATACOMUNICACAO'].astype('datetime64')

In [ ]:
dias = (dataset['DATACOMUNICACAO'] - dataset['DATAOCORRENCIA'])/ np.timedelta64(1, 'D')

In [ ]:
dataset.head(15)

In [ ]:
dias.mean()

In [ ]:
dias.nlargest(1)

### Análise gráfica dos dados

#### Capturando nome das colunas importantes no dataset

In [ ]:
string_gerada=""
for item in dataset.columns:
    string_gerada += "'"+item+"'"+", "
string_gerada

In [ ]:
drop_c = ['ANO_BO', 'NUM_BO', 'BO_INICIADO', 'BO_EMITIDO', 'DATAOCORRENCIA', 'HORAOCORRENCIA', 'PERIDOOCORRENCIA', 'DATACOMUNICACAO', 'DATAELABORACAO', 'BO_AUTORIA', 'FLAGRANTE', 'LOGRADOURO', 'NUMERO', 'BAIRRO', 'CIDADE', 'UF', 'LATITUDE', 'LONGITUDE', 'DESCRICAOLOCAL', 'SOLUCAO', 'DELEGACIA_NOME', 'DELEGACIA_CIRCUNSCRICAO', 'ESPECIE', 'RUBRICA', 'DESDOBRAMENTO', 'STATUS', 'QUANT_CELULAR', 'MARCA_CELULAR']

In [ ]:
string_gerada=[]
for item in dataset.columns:
    string_gerada.append(item)
string_gerada

In [ ]:
columns = ['NUMERO_BOLETIM', 'NUMERO_BOLETIM_PRINCIPAL', 'EXAME', 'NOMEPESSOA', 'TIPOPESSOA', 'VITIMAFATAL', 'RG', 'RG_UF', 'NATURALIDADE', 'NACIONALIDADE', 'SEXO', 'DATANASCIMENTO', 'IDADE', 'ESTADOCIVIL', 'PROFISSAO', 'GRAUINSTRUCAO', 'CORCUTIS', 'NATUREZAVINCULADA', 'TIPOVINCULO', 'RELACIONAMENTO', 'PARENTESCO', 'PLACA_VEICULO', 'UF_VEICULO', 'CIDADE_VEICULO', 'DESCR_COR_VEICULO', 'DESCR_MARCA_VEICULO', 'ANO_FABRICACAO', 'ANO_MODELO', 'DESCR_TIPO_VEICULO']

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
